In [1]:
import json
from sklearn.manifold import TSNE

from loading_preprocessing_TC import *
DATASET_PATH = 'resources/datasets/semeval/train/'
files = [DATASET_PATH + 'SemEval2016-Task3-CQA-QL-train-part1-subtaskA.xml',
             DATASET_PATH + 'SemEval2016-Task3-CQA-QL-train-part2-subtaskA.xml']
train_xml = read_xml(files)
train, answer_texts_train = xml2dataframe_Labels(train_xml, 'train')
answer_texts_train.set_index('answer_id', drop=False, inplace=True)

# qa_pairs, answer_texts = load_data()

Picking GPU 0
Threads:  1790
Questions:  1790
Comments:  17900


In [2]:
qa_pairs = train
print(qa_pairs)

                                                   question  \
Q1_R1     massage oil is there any place i can find scen...   
Q1_R6     Philipino Massage center Hi;Can any one tell m...   
Q1_R8     Best place for massage Tell me; where is the b...   
Q1_R10    body massage hi there; i can see a lot of mass...   
Q1_R22    What attracts you more ? What attracts you more ?   
Q1_R25    You have got to be joking.....Seen in a shop i...   
Q1_R27    Blackheads Any suggestions on how to get rid o...   
Q1_R32    Where to get Tea Tree Oil Can someone please a...   
Q1_R43    "very strong migraine pain what to do ? plz he...   
Q1_R46    --Garlic Oil-- Can someone please tell me wher...   
Q2_R20    Convert Tourist visa to Family visit visa Hi a...   
Q2_R21    Maximum period of a Visit Visa? I'm currently ...   
Q2_R32    Re-apply family visit visa (no need to wait 3 ...   
Q2_R46    Visa Extension for My Sister in Law Dear All; ...   
Q2_R60    Family visit visa extension in Imigration? De

In [3]:
answer_texts = answer_texts_train
print(len(answer_texts))

17900


In [4]:
import pickle
from keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 200

DATA_PATH = 'out/data/semeval/'
with open(DATA_PATH + 'tokenizer.p', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(DATA_PATH + 'embedding_matrix.p', 'rb') as handle:
    embeddings = pickle.load(handle)

def prepare_data(texts):
    """Tokenize texts and pad resulting sequences of words using Keras functions."""
    global tokenizer, embeddings
    tokens = tokenizer.texts_to_sequences(texts)
    padded_tokens = pad_sequences(tokens, maxlen=MAX_LENGTH, value=embeddings.shape[0] - 1)
    return tokens, padded_tokens


Using TensorFlow backend.


In [13]:
perplexity = 20
print(len(qa_pairs))
all_questions = []
for i in range(len(qa_pairs)):
    current_row = qa_pairs.iloc[i]
    question = current_row['question']
    # print(question)
    q_tokens, q_padded_tokens = prepare_data([question])
    # print('q_tokens', q_tokens[0])
    # print('q_padded_tokens', q_padded_tokens[0])
    # print(q_tokens, q_padded_tokens)
    all_questions.append(q_padded_tokens[0])
# question_vector = rnn_values[0]
# print(question_vector)
# print(all_questions)
print('TSNE on all questions...')
tsne_model = TSNE(perplexity=perplexity, n_components=2, init='pca', n_iter=2500, random_state=23,
                      metric="cosine")
new_values = tsne_model.fit_transform(all_questions)
print('Finished!')
print(new_values)


1790
TSNE on all questions...
Finished!
[[-116800.47    58959.984]
 [ -12122.51   -54090.035]
 [ -37651.28   -47161.92 ]
 ...
 [-110689.164   50524.766]
 [-113138.95    54886.64 ]
 [-117892.77    63516.04 ]]


In [14]:
trace_question_x = []
trace_question_y = []
trace_question_text = []
trace_question_hovertext = []

for i in range(len(qa_pairs)):
    question = qa_pairs.iloc[i]['question']
    trace_question_x.append(new_values[i][0])
    trace_question_y.append(new_values[i][1])
    trace_question_text.append('Q' + str(i))
    trace_question_hovertext.append(question if len(question) < 61 else question[:60] + '...')

marker_blue = {
        'size': 20,
        'color': 'rgb(0, 0, 255)',
        # star
        'symbol': 17
    }
trace_question = {
        'name': 'Question',
        'x': trace_question_x,
        'y': trace_question_y,
        'type': 'scatter',
        'mode': 'markers+text',
        'hoverinfo': 'text',
        'hovertext': trace_question_hovertext,
        'text': trace_question_text,
        'textposition': 'top right',
        'marker': marker_blue
    }

plotly_tsne = [trace_question]
plotly_tsne_as_json = pd.Series(plotly_tsne).to_json(orient='values')

# print(plotly_tsne_as_json)
print('Writing to file...')
with open('out/data/plotly_all_questions_json_string_p20.json', 'w') as outfile:  
    json.dump(plotly_tsne_as_json, outfile)
    print('Finished!')

Writing to file...
Finished!
